# Gesture Recognition with Hidden Markov Models
## HMM Filters
### Forward Filter

In [74]:
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
import pickle as pkl
from sklearn.cluster import KMeans

In [75]:
def forward_filt(A, B, pi, obs):
    #q_ki = P(s_k = i, y_1:k)
    alpha = np.zeros((len(obs),len(A))) #n_obs x n_states
    scale_factor = np.zeros(len(obs))

    #initialization
    o_k = obs[0] #scalar observation at time step k
    B_k = np.diag(B[:,o_k]) #4x4 diagonal matrix
    alpha[0] = np.dot(B_k,pi) #4x1 vector
    alpha[0][alpha[0] == 0] = 1e-300
    #scale_factor[0] = 1/np.sum(alpha[0])
    #alpha[0] = alpha[0] * scale_factor[0]#/np.sum(alpha[0])
    alpha[0] = alpha[0]/np.sum(alpha[0])
    for k in range(1,len(obs)):
        o_k = obs[k]
        B_k = np.diag(B[:,o_k])
        alpha[k] = np.dot(np.dot(B_k,A),alpha[k-1]) #alpha[k] = B_k * A * alpha[k-1]
        alpha[k][alpha[k] == 0] = 1e-300
        # scale_factor[k] = 1/np.sum(alpha[k])
        # alpha[k] = alpha[k] * scale_factor[k]
        alpha[k] = alpha[k]/np.sum(alpha[k]) #normalize
    return [alpha, scale_factor]

### Backward Filter

In [76]:
def backward_filt(A, B, obs, scale_factor):
    K = len(obs)
    beta = np.zeros((K,len(A)))
    #initialization
    beta[K-1] = np.ones(len(A))
    beta[K-1] = beta[K-1]/len(A)
    #beta[K-1] = beta[K-1] * scale_factor[K-1]
    for k in range(K-1,0,-1):
        o_k = obs[k]
        B_k = np.diag(B[:,o_k]) #diagonalize for elementwise multiplication w/ dot prod
        beta[k-1] = np.dot(np.dot(A,B_k),beta[k]) #beta[k-1] = A * B_k * beta[k]
        beta[k-1][beta[k-1] == 0] = 1e-300
        beta[k-1] = beta[k-1]/np.sum(beta[k-1])
        #beta[k-1] = beta[k-1] * scale_factor[k-1]
    return beta


### Smoother

In [77]:
def smoother(alpha, beta):
    K = len(alpha) #number of observations/time steps
    gamma = np.zeros(alpha.shape)
    for k in range(K):
        alpha_k = alpha[k]
        beta_k = beta[k]
        num = np.multiply(alpha_k, beta_k)
        den = np.sum(num) #sum across all states
        gamma[k] = num/den #normalize
    return gamma

### Paired States

In [78]:
def paired_st(A, B, alpha, beta, obs):
    K = len(alpha) #number of observations
    X = alpha.shape[1] #number of states
    xi = np.zeros([K-1,X,X])
    for k in range(K-1):
        alpha_k = alpha[k,:,np.newaxis]
        beta_k1 = beta[k+1,:]

        o_k1 = obs[k+1]
        B_k1 = B[:,o_k1]

        xi[k] = alpha_k * A * B_k1 * beta_k1
        xi[k][xi[k] == 0] = 1e-300
        xi[k] = xi[k]/np.sum(xi[k]) #normalize
    return xi

## Baum-Welsch Algorithm

In [79]:
def baum_welsch(HMM, obs, N): #(initial guesses,,, observations, number of iterations)
    A_est = np.copy(HMM[0])
    B_est = np.copy(HMM[1])
    pi_est = np.copy(HMM[2])
    for n in range(N):
        #E-Step---------------
        #HMM Filtering
        #---------------------
        alpha, c = forward_filt(A_est, B_est, pi_est, obs) #run forward filter
        beta = backward_filt(A_est, B_est, obs, c) #run backward filter
        gamma = smoother(alpha, beta) #run smoother
        xi = paired_st(A_est, B_est, alpha, beta, obs)

        #M-Step---------------
        #update parameter estimates
        #---------------------
        #update pi_est
        pi_est = gamma[0]

        #update A_est
        A_num = np.sum(xi,axis=0) #expected num of jumps from i to j
        A_den = np.sum(gamma[0:-1],axis=0)[:,np.newaxis] #total expected visits to i
        A_den[A_den == 0] = 1e-300
        A_est = A_num/A_den

        #update B_est
        B_den = np.sum(gamma,axis=0)
        B_den[B_den == 0] = 1e-300
        for o in range(B_est.shape[1]): #loop through all possible observations
            B_num = np.sum(gamma[obs == o],axis=0) #sum over observations o 
            B_est[:,o] = B_num/B_den

    return [A_est, B_est, pi_est]

## K-Means Algorithm

In [70]:
class MyKMeans:
    def __init__(self, n_clusters, cutoff=0.05, max_iter = 500):
        self.n_clusters = n_clusters
        self.centroids = np.zeros(n_clusters)
        self.cutoff = cutoff
        self.max_iter = max_iter

    def _sos(self, arr): #sum of squares
        return np.sum(np.power(arr,2),axis=tuple(range(1,arr.ndim)))

    #CREDIT: Eli told me about this trick to calculate distances on the entire arrays
    #        to avoid looping though every element
    def _dist_arrs(self, X1, X2): #l2 norm between vectors in 2 arrays
        x1_sos = np.expand_dims(self._sos(X1), axis=1)
        x2_sos = self._sos(X2)
        dot_sos = -2*X1.dot(X2.T)
        sos = np.round(x1_sos+x2_sos+dot_sos,10) #round to 10 decimals to avoid negative due to machine error
        return np.sqrt(sos)

    def _dist(self,x1,x2): #l2 norm between 2 vectors
        sq_err = np.power((x1 - x2),2)
        sm = np.sum(sq_err)
        return np.sqrt(sm)

    def _calc_centroids(self, X, labels): #calculate new centroids as mean position of all x with same label
        n_cents = np.array([np.mean(X[labels == i],axis=0) for i in range(self.n_clusters)])
        n_cents[np.isnan(n_cents)] = self.centroids[np.isnan(n_cents)] #any centroids with no members remain stationary
        return np.array([np.mean(X[labels == i],axis=0) for i in range(self.n_clusters)])
    
    def _nearest_centroid_arr(self, X): #calculate nearest centroid for each element in array
        dist = self._dist_arrs(X,self.centroids)
        labels = np.argmin(dist,axis=1) #label with index of nearest centroid
        return labels

    def _nearest_centroid(self, x):
        min_dist = 1e100
        label = 0
        for i,cent in enumerate(self.centroids):
            d = self._dist(x,cent)
            if(d < min_dist):
                label = i
                min_dist = d
        # dist = [self._dist(x,cent) for cent in self.centroids] #calculate distances to centroids
        # label = np.argmin(dist) #label with index of closest centroid
        return label

    def fit(self,X):
        X = np.array(X)
        cent_idx = np.random.choice(range(len(X)), self.n_clusters, replace=False) #indices of x's that will be used as initial centroids
        self.centroids = np.array([X[i] for i in cent_idx])
        xlabels = self._nearest_centroid_arr(X) #label each sample with nearest centroid
        training = True
        cnt = 0
        while training:
            cnt = cnt + 1
            n_centroids = self._calc_centroids(X, xlabels) #new centroids at mean of labeled samples
            diff = np.max([self._dist(n_centroids[i], centroid) for i,centroid in enumerate(self.centroids)]) #distance centroids moved
            self.centroids = n_centroids #update centroids
            xlabels = self._nearest_centroid_arr(X) #label each sample x by nearest centroid
            training = diff > self.cutoff #exit loop if no centroid moves by more than cutoff
            if(cnt > self.max_iter):
                print('Maximum Iterations',self.max_iter,'reached before cunvergence')
                training = False
        return self

    def predict(self,x):
        if(x.ndim > 1):
            return self._nearest_centroid_arr(x)
        else:
            return self._nearest_centroid(x)

## Training Models
### Import Data

In [5]:
TRAIN_DIR_PATH = '../train/'

train_files = listdir(TRAIN_DIR_PATH)

#CREDIT: Eli gave me the idea to use a dictionary like this rather
#        than the messy attempt I had at just using arrays
keys = ['wave','inf','eight','circle','beat3','beat4']
obs_dict = {k:[] for k in keys}
for f in train_files:
    obs = np.genfromtxt(fname=TRAIN_DIR_PATH + f, usecols=[1,2,3,4,5,6]) #read observations from file
    if('wave' in f):
        obs_dict['wave'].append(obs)
    elif('inf' in f):
        obs_dict['inf'].append(obs)
    elif('eight' in f):
        obs_dict['eight'].append(obs)
    elif('circle' in f):
        obs_dict['circle'].append(obs)
    elif('beat3' in f):
        obs_dict['beat3'].append(obs)
    elif('beat4' in f):
        obs_dict['beat4'].append(obs)

### Cluster Observations using KMeans

In [71]:
nstates = 20 #number of possible states
nobs = 100 #number of possible observations

kmeans = {}
c_obs_dict = {}
for k in keys:
    kmeans[k] = MyKMeans(n_clusters=nobs,cutoff=0.1).fit(np.concatenate(obs_dict[k])) #train k-means
    c_obs_dict[k] = [kmeans[k].predict(obs) for obs in obs_dict[k]] #classified/clustered observations

In [22]:
nstates = 20 #number of possible states
nobs = 100 #number of possible observations

kmeans = {}
c_obs_dict = {}
for k in keys:
    kmeans[k] = KMeans(n_clusters=nobs, random_state=0).fit(np.concatenate(obs_dict[k])) #train k-means
    c_obs_dict[k] = [kmeans[k].predict(obs) for obs in obs_dict[k]] #classified/clustered observations

### Pickle KMeans

In [72]:
#store trained kmeans in pkl because they take forever to train
fname = 'kmeans.pkl'
f = open(fname,'wb')
pkl.dump(kmeans,f)
f.close()

### Load Pickled KMeans

In [3]:
#Read pickled kmeans file to avoid retraining models
fname = 'kmeans.pkl'
f = open(fname,'rb')
kmeans = pkl.load(f)
f.close

<function BufferedReader.close>

### Train HMMs

In [80]:
#generate random probability matrix (rows sum to 1)
def rand_prob_mat(shape):
    p = np.zeros(shape)
    nrows = shape[0]
    if(len(shape) == 1):
        p = np.random.rand(nrows)
        p = p/np.sum(p)
    elif(len(shape) == 2):
        ncols = shape[1]
        for i in range(nrows):
            p[i,:] = np.random.rand(1,ncols)
            p[i,:] = p[i,:]/np.sum(p[i,:])
    return p

In [81]:
N = 50 #number of iterations for Baum-Welsch

A_rand = rand_prob_mat((nstates,nstates))
B_rand = rand_prob_mat((nstates,nobs))
pi_rand = rand_prob_mat((nstates,))

HMMs = {}
for k in keys:
    print("training",k,end='')
    HMMs[k] = [A_rand, B_rand, pi_rand] #randomly initialize HMM
    for c_obs in c_obs_dict[k]: #loop through observation files
        HMMs[k] = baum_welsch(HMMs[k], c_obs, N) #train HMM
        print('.',end='')
    print('done')
    for param in HMMs[k]:
        assert not np.any(np.isnan(param)) #error if there are any nans
print('Done!')

training wave.....done
training inf.....done
training eight.....done
training circle.....done
training beat3.....done
training beat4.....done
Done!


### Pickle Trained HMMs

In [87]:
#store trained HMMs in pkl because they take forever to train
fname = 'HMMs.pkl'
f = open(fname,'wb')
pkl.dump(HMMs,f)
f.close()

### Load Pickled HMMs

In [30]:
#Read pickled HMM file to avoid retraining models
fname = 'HMMs.pkl'
f = open(fname,'rb')
HMMs = pkl.load(f)
f.close

<function BufferedReader.close>

## Predict Gestures

In [84]:
def obs_mod_lik(HMM, obs): #likelihood that given observations came from given HMM
    A, B, pi = HMM
    alpha, c = forward_filt(A, B, pi, obs) #forward filter
    beta = backward_filt(A, B, obs, c) #backward filter
    lik = np.sum(beta[0] * pi) #calculate probability of seeing these obs under this HMM
    return lik

In [85]:
def classify_gesture(HMMs, obs):
    keys = np.array(list(HMMs.keys()))
    l = np.zeros(len(keys))
    max_l = 0
    pred = 'none'
    for i,k in enumerate(keys):
        c_obs = kmeans[k].predict(obs)
        l[i] = obs_mod_lik(HMMs[k], c_obs)
    # arrange likelihoods and keys in descending order
    sort_ind = l.argsort()
    sorted_l = l[sort_ind[::-1]]
    sorted_keys = keys[sort_ind[::-1]]
    return [sorted_keys,sorted_l] #returns gestures in order of decreasing likelihood and associated likelihoods

## Test

In [86]:
TEST_DIR_PATH = '../old_test/'
test_files = listdir(TEST_DIR_PATH)

for f in test_files:
    obs = np.genfromtxt(fname=TEST_DIR_PATH + f, usecols=[1,2,3,4,5,6]) #read observations from file
    pred,ls = classify_gesture(HMMs, obs)
    print('----',f,'----')
    for i in range(3):
        print(i+1, '. ', pred[i], '\t', ls[i], sep='')
    print()

---- beat3_31.txt ----
1. beat4	0.3132756053324105
2. wave	0.05
3. circle	0.04999999999999999

---- beat4_31.txt ----
1. wave	0.05
2. beat4	0.04999999999999999
3. beat3	0.04999999999999999

---- circle31.txt ----
1. circle	0.04999999999999999
2. eight	0.04999999999999999
3. inf	0.04999999999999999

---- eight31.txt ----
1. wave	0.05
2. eight	0.04999999999999999
3. inf	0.04999999999999999

---- inf31.txt ----
1. wave	0.05
2. eight	0.04999999999999999
3. inf	0.04999999999999999

---- wave31.txt ----
1. beat4	0.3132756053324105
2. eight	0.04999999999999999
3. inf	0.04999999999999999

